In [1]:
import csv
import cv2
import numpy as np

#read csv file and load it to list
lines=[]
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader,None) #for skipping header
    #i=0
    for line in reader:
        lines.append(line)
        #print(line)
        #i += 1
        #if(i>20):
         #   break
        
images = []
measurements = []
#i=0
for line in lines:
    
    #i += 1
    #if(i>20):
    #    break
    source_path_center = line[0]
    source_path_left = line[1]
    source_path_right = line[2]
    filename_center = source_path_center.split('/')[-1]
    filename_left = source_path_left.split('/')[-1]
    filename_right = source_path_right.split('/')[-1]
    #print(line[3])
    current_path_center = 'data/IMG/'+ filename_center
    current_path_left = 'data/IMG/'+filename_left
    current_path_right = 'data/IMG/'+filename_right
    #print(current_path_left)
    image_center = cv2.imread(current_path_center)
    image_left = cv2.imread(current_path_left)
    image_right = cv2.imread(current_path_right)
    assert image_center is not None
    assert image_left is not None
    assert image_right is not None
    
    images.append(image_center)
    images.append(cv2.flip(image_center,1))
    measurement_center = float(line[3])
    measurements.append(measurement_center)
    measurements.append(measurement_center*-1.0)
    
    #adding left and right images
    correction = 0.2
    measurement_left = measurement_center + correction
    measurement_right = measurement_center - correction
    images.append(image_left)
    images.append(cv2.flip(image_left,1))
    measurements.append(measurement_left)
    measurements.append(measurement_left*-1.0)
    
    images.append(image_right)
    images.append(cv2.flip(image_right,1))
    measurements.append(measurement_right)
    measurements.append(measurement_right*-1.0)
    
    
X_train = np.array(images)
y_train = np.array(measurements)

n_num = X_train[3].shape

print("shape", n_num)
    

shape (160, 320, 3)


In [2]:
#define network

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda,Convolution2D,MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x:x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
model.fit(X_train,y_train,validation_split=0.2,shuffle=True,nb_epoch=5)

model.save('model.h5')




Using TensorFlow backend.


Train on 10156 samples, validate on 2540 samples
Epoch 1/5
10156/10156 [==============================] - 321s - loss: 0.7144 - val_loss: 0.0558